# Data preprocessing block

## Spliting Data
Split the data into 90% for training, 10% for validation.

In [1]:
import os
from sklearn.model_selection import train_test_split
total_list = os.listdir('/content/drive/My Drive/NCTU-1091深度學習/HW1/dataset/training_data/training_data')
train_list, val_list = train_test_split(total_list, random_state=10, train_size=0.9)

## Organize Dataset
Create image corresponding class map dictionary.

In [4]:
import csv
with open('/content/drive/My Drive/NCTU-1091深度學習/HW1/dataset/training_labels.csv', newline='') as csvfile:
  total_dict = {}
  rows = csv.reader(csvfile)
  index = next(rows)
  for row in rows:
    if row[1] not in total_dict:
      total_dict[row[1]] = []
      total_dict[row[1]].append(row[0])
    else:
      total_dict[row[1]].append(row[0])

Move images of the same class into same train or validation class folder.

In [20]:
import shutil
target_path = '/content/drive/My Drive/NCTU-1091深度學習/HW1/PMG-Progressive-Multi-Granularity-Training/bird/'
src_path = '/content/drive/My Drive/NCTU-1091深度學習/HW1/dataset/training_data/training_data/'
for class_name in total_dict.keys():
  if os.path.exists(r'/content/drive/My Drive/NCTU-1091深度學習/HW1/PMG-Progressive-Multi-Granularity-Training/bird/train/{}'.format(class_name)):
    continue
  if '/' in class_name:
    print('this class is illegal:', class_name)
    for img_number in total_dict[class_name]:
      img_number = img_number+'.jpg'
      if img_number in train_list:
        shutil.copy(src_path+img_number, '/content/drive/My Drive/NCTU-1091深度學習/HW1/PMG-Progressive-Multi-Granularity-Training/bird/train/Ram C V Cargo Van Minivan 2012/'+img_number)
      else:
        shutil.copy(src_path+img_number, '/content/drive/My Drive/NCTU-1091深度學習/HW1/PMG-Progressive-Multi-Granularity-Training/bird/test/Ram C V Cargo Van Minivan 2012/'+img_number)
    print(class_name, 'done!')
    continue
  os.mkdir(r'/content/drive/My Drive/NCTU-1091深度學習/HW1/PMG-Progressive-Multi-Granularity-Training/bird/train/{}'.format(class_name))
  os.mkdir(r'/content/drive/My Drive/NCTU-1091深度學習/HW1/PMG-Progressive-Multi-Granularity-Training/bird/test/{}'.format(class_name))
  for img_number in total_dict[class_name]:
    img_number = img_number+'.jpg'
    if img_number in train_list:
      shutil.copy(src_path+img_number, target_path+r'train/{}'.format(class_name+'/'+img_number))
    else:
      shutil.copy(src_path+img_number, target_path+r'test/{}'.format(class_name+'/'+img_number))
  print(class_name, 'done!')

this class is illegal: Ram C/V Cargo Van Minivan 2012
Ram C/V Cargo Van Minivan 2012 done!


# Training block

## Train model
Train model and save it.

In [1]:
%cd /content/drive/My Drive/NCTU-1091深度學習/HW1/PMG-Progressive-Multi-Granularity-Training


/content/drive/My Drive/NCTU-1091深度學習/HW1/PMG-Progressive-Multi-Granularity-Training


In [3]:
!python train.py


True
==> Preparing data..
/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:257: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")
==> Building model..

Epoch: 0
Step: 0 | Loss1: 5.377 | Loss2: 5.48127 | Loss3: 5.30428 | Loss_concat: 11.07520 | Loss: 27.238 | Acc: 0.000% (0/16)
Step: 50 | Loss1: 5.312 | Loss2: 5.41644 | Loss3: 5.29225 | Loss_concat: 10.57218 | Loss: 26.593 | Acc: 2.206% (18/816)
Step: 100 | Loss1: 5.266 | Loss2: 5.32563 | Loss3: 5.15033 | Loss_concat: 10.13082 | Loss: 25.872 | Acc: 4.208% (68/1616)
Step: 150 | Loss1: 5.199 | Loss2: 5.22773 | Loss3: 4.99455 | Loss_concat: 9.61480 | Loss: 25.036 | Acc: 6.581% (159/2416)
Step: 200 | Loss1: 5.131 | Loss2: 5.10550 | Loss3: 4.82906 | Loss_concat: 9.07416 | Loss: 24.139 | Acc: 9.981% (321/3216)
Step: 250 | Loss1: 5.080 | Loss2: 4.97664 | Loss3: 4.66054 | Loss_concat: 8.55027 | Loss: 23.267 | Acc: 13.6

# Inference block

## Build class index map
the classes.txt is build using method of torchvision.datasets.ImageFolder.class_to_idx

In [1]:
with open('/content/drive/My Drive/NCTU-1091深度學習/HW1/PMG-Progressive-Multi-Granularity-Training/bird/classes.txt', 'r') as f:
  classes = f.read().split('\n')
  clidx_map = {}
  for i in classes:
    map = i.split(',')
    if len(map) > 1:
      clidx_map[map[1]] = map[0]
clidx_map

{'0': 'AM General Hummer SUV 2000',
 '1': 'Acura Integra Type R 2001',
 '10': 'Aston Martin Virage Coupe 2012',
 '100': 'Ferrari 458 Italia Convertible 2012',
 '101': 'Ferrari 458 Italia Coupe 2012',
 '102': 'Ferrari California Convertible 2012',
 '103': 'Ferrari FF Coupe 2012',
 '104': 'Fisker Karma Sedan 2012',
 '105': 'Ford E-Series Wagon Van 2012',
 '106': 'Ford Edge SUV 2012',
 '107': 'Ford Expedition EL SUV 2009',
 '108': 'Ford F-150 Regular Cab 2007',
 '109': 'Ford F-150 Regular Cab 2012',
 '11': 'Audi 100 Sedan 1994',
 '110': 'Ford F-450 Super Duty Crew Cab 2012',
 '111': 'Ford Fiesta Sedan 2012',
 '112': 'Ford Focus Sedan 2007',
 '113': 'Ford Freestar Minivan 2007',
 '114': 'Ford GT Coupe 2006',
 '115': 'Ford Mustang Convertible 2007',
 '116': 'Ford Ranger SuperCab 2011',
 '117': 'GMC Acadia SUV 2012',
 '118': 'GMC Canyon Extended Cab 2012',
 '119': 'GMC Savana Van 2012',
 '12': 'Audi 100 Wagon 1994',
 '120': 'GMC Terrain SUV 2012',
 '121': 'GMC Yukon Hybrid SUV 2012',
 '122':

## Load testing filename list

In [ ]:
import os
img_path = '/content/drive/My Drive/NCTU-1091深度學習/HW1/dataset/testing_data'
imgidlist = os.listdir(img_path+'/testing_data')
for i in range(len(imgidlist)):
  imgidlist[i] = int(imgidlist[i].rstrip('.jpg'))
imgidlist.sort()
for j in range(len(imgidlist)):
  imgidlist[j] = str(imgidlist[j]).rjust(6, '0')
print(imgidlist)


## Load the model and do batch inference and write the corresponding filename and class into csv file.

In [3]:
import numpy as np
import csv
import random
import torch
import torchvision
from torch.autograd import Variable
from torchvision import transforms, models
import torch.nn.functional as F
%cd '/content/drive/My Drive/NCTU-1091深度學習/HW1/PMG-Progressive-Multi-Granularity-Training'
model_path = '/content/drive/My Drive/NCTU-1091深度學習/HW1/PMG-Progressive-Multi-Granularity-Training/bird/model.pth'
batch_size = 20
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0")
net = torch.load('/content/drive/My Drive/NCTU-1091深度學習/HW1/PMG-Progressive-Multi-Granularity-Training/bird/model.pth').to(device)
net.eval()
transform_test = transforms.Compose([
    transforms.Resize((550, 550)),
    transforms.CenterCrop(448),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])
testset = torchvision.datasets.ImageFolder(root=img_path, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=1)
mark = 0
with torch.no_grad():
  with open('/content/drive/My Drive/NCTU-1091深度學習/HW1/PMG-Progressive-Multi-Granularity-Training/bird/predictions2.csv', 'w', newline='') as csf:
    writer = csv.writer(csf)
    writer.writerow(['id', 'label'])
    for batch_idx, (inputs, targets) in enumerate(testloader):
      if use_cuda:
          inputs = inputs.to(device)
      inputs = Variable(inputs)
      output_1, output_2, output_3, output_concat = net(inputs)
      outputs_com = output_1 + output_2 + output_3 + output_concat
      _, predicted_com = torch.max(outputs_com.data, 1)
      for i in range(mark, mark+len(predicted_com)):
        writer.writerow([imgidlist[i], clidx_map[str(predicted_com.cpu().numpy()[i % batch_size])]])
      print('batch ', batch_idx, ' complete!')
      mark += batch_size


/content/drive/My Drive/NCTU-1091深度學習/HW1/PMG-Progressive-Multi-Granularity-Training
batch  0  complete!
batch  1  complete!
batch  2  complete!
batch  3  complete!
batch  4  complete!
batch  5  complete!
batch  6  complete!
batch  7  complete!
batch  8  complete!
batch  9  complete!
batch  10  complete!
batch  11  complete!
batch  12  complete!
batch  13  complete!
batch  14  complete!
batch  15  complete!
batch  16  complete!
batch  17  complete!
batch  18  complete!
batch  19  complete!
batch  20  complete!
batch  21  complete!
batch  22  complete!
batch  23  complete!
batch  24  complete!
batch  25  complete!
batch  26  complete!
batch  27  complete!
batch  28  complete!
batch  29  complete!
batch  30  complete!
batch  31  complete!
batch  32  complete!
batch  33  complete!
batch  34  complete!
batch  35  complete!
batch  36  complete!
batch  37  complete!
batch  38  complete!
batch  39  complete!
batch  40  complete!
batch  41  complete!
batch  42  complete!
batch  43  complete!
b